# Exploration des cahiers de doléance

In [43]:
import re
import csv
import spacy
from collections import defaultdict

In [6]:
nlp = spacy.load('fr_core_news_sm')

In [2]:
with open('./resources/cahiers.csv') as f:
    data = list(csv.DictReader(f))

In [15]:
for item in data:
    doc = nlp(item['texte_cahier_txt'])
    item['sentences'] = [str(sent) for sent in doc.sents]

In [31]:
QUERY = re.compile(r'\bsel\b')

for item in data:
    for sent in item['sentences']:
        if QUERY.search(sent):
            print('%s (%s):' % (item['localite'], item['rural_littoral']), sent)
            print()

Marennes (littoral): — Que dans le cas où le régime actuel de la gabelle seroit changé, comme il y a lieu de le présumer, les salines de Saintonge ne pouvant se soutenir qu’autant que les sels qui en proviennent auront un débouché assuré, le bon plaisir de Sa Majesté soit que les pays actuellement aprovisionnés des sels de nos salines, dont la qualité supérieure est générallement reconnue, seront tenus d’y prendre le sel nécessaire à leur consommation et de constater le chargement de leur navires dans nos ports par un certiftîcat du bureau de l’amirauté et autres expéditions des navires, l’objet de cette demande étant d’assurer la conservation de cette précieuse saline et aux consommateurs des sels de la meilheure qualité.

Marennes (littoral): — Qu’il soit ordonné, par la même raison, aux armateurs pour la pêche du poisson salé, de continuer à prendre le sel nécessaire pour leur pêche dans les marais de Brouage, ainsi qu’ils y sont obligés par l’article 1er chi titre 15 de l’ordonnanc

In [45]:
doc = nlp(data[1]['texte_cahier_txt'])
entities = defaultdict(set)
for ent in doc.ents:
    if len(ent.text) < 4 or len(ent.text) > 40:
        continue
    entities[ent.label_].add(ent.text.replace('\xa0', ''))

entities

defaultdict(set,
            {'ORG': {'Assemblée',
              'CAHIER',
              'Conseil',
              'DOLÉANCES',
              'Lieutenant général de Niort',
              'Tiers',
              'Tiers état de la Sénéchaussée de Niort',
              'l’Assemblée',
              'l’Assemblée des',
              'l’Assemblée générale',
              'l’Etat'},
             'PER': {'Auguste',
              'Auguste Monarque',
              'Date',
              'Députés',
              'Etats Généraux',
              'Jean Basque',
              'Monsieur',
              'Pierre Falignon',
              'Population',
              'Président',
              'Sa Majesté',
              'Sénéchaussée',
              'l’Élection'},
             'LOC': {'Bazauges',
              'Clergé',
              'Elections',
              'Etat',
              'Etats provinciaux',
              'Marrais',
              'Noblesse',
              'Rois',
              'Royaume',
          